In [2]:
import numpy as np
import pandas as pd


In [3]:
data = pd.read_csv("airfoil_noise_data.csv")
data.head()

,x0,x1,x2,x3,x4,y
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [4]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        # for leaf node
        self.value = value

In [5]:
class DecisionTreeRegressor():
    def __init__(self, min_samples_split=2, max_depth=5):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["var_red"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_var_red = -float("inf")
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    if curr_var_red > max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
        return best_split

    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right

    def variance_reduction(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
        return reduction

    def calculate_leaf_value(self, Y):
        return np.mean(Y)

    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
        if tree.value is not None: 
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

    def predict(self, X):
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

In [6]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=41)

regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=5)
regressor.fit(X_train, Y_train)
regressor.print_tree()

Y_pred = regressor.predict(X_test)

rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))
print("Root Mean Squared Error:", rmse)

X_0 <= 3150.0 ? 7.132048702017748
 left:X_4 <= 0.0337792 ? 3.590330569067664
  left:X_3 <= 55.5 ? 1.17898999813184
    left:X_4 <= 0.00251435 ? 1.614396721819876
        left:X_0 <= 800.0 ? 7.309805085234405
                left:X_1 <= 3.3 ? 6.7559488725981875
                                left:125.53021739130433
                                right:119.11366666666665
                right:X_2 <= 0.1016 ? 1.4617094982299497
                                left:131.15992957746477
                                right:128.2403
        right:X_0 <= 1600.0 ? 4.676816126979858
                left:X_0 <= 400.0 ? 2.7647419932246002
                                left:124.58287000000001
                                right:128.20404329004327
                right:X_2 <= 0.0508 ? 6.571240886308157
                                left:125.8051875
                                right:120.28059999999999
    right:X_1 <= 15.4 ? 2.2342245360792994
        left:X_2 <= 0.1016 ? 2.09506677858699

In [10]:
regressor = DecisionTreeRegressor(min_samples_split=2, max_depth=20)
regressor.fit(X, Y)


In [11]:
Y_pred_train = regressor.predict(X)

In [14]:
from sklearn.metrics import r2_score, mean_squared_error

r2_train = r2_score(Y, Y_pred_train)
rmse_train = np.sqrt(mean_squared_error(Y, Y_pred_train))

print("Training Root Mean Squared Error:", rmse_train)
print("Training Accuracy: {:.2f}%".format(r2_train * 100))

Training Root Mean Squared Error: 0.0
Training Accuracy: 100.00%
